In [ ]:
library(knitr)
opts_chunk$set(warning = FALSE, message = FALSE, cache = TRUE, autodep = TRUE, tidy = FALSE)
library("rgl")
knit_hooks$set(webgl = hook_webgl)



# Dimension Reduction

As soon as the dimension is larger than 3, it becomes hard to visualize the raw data. Dimension reduction technique can be used to alleviate this issue by *projecting* the data in a low dimensional space, typically a 2D space. Note that those dimension reduction ideas can also be used a preprocessing step for any learning task.



## Principal Component Analysis

We will start by the most classical dimension reduction algorithm, the Principal Component Analysis one. The idea is to find a subspace spanned by $d'$ orthonormal columns $V^{(l)}$ such that the reconstruction error between the data and its projection on this subspace
\[
\sum_{i=1}^n \| \textbf{X}_i - m + V^t (\textbf{X}_i -m) V\|^2
\]
 is as small as possible. An explicit solution to this problem is given by the space spanned by the $d'$ eigenvectors of the $d \times d$ empirical covariance matrix of the observations.

The ACP is available in many package and will use the one provided by __FactoMineR__.

### Decathlon dataset

We will first consider a _classical_ dataset __decathlon__. It contains the performance of several athlets during two decathlons in 2004 the Decastar and the Olympic Game. Each observations consists of

- the 10 raw performance in the 10 events,
- the ranking in the event,
- the total number of points,
- the name of the event.

During this lab, we will mainly focus on the 10 first columns.



In [ ]:
library("FactoMineR")
library("dplyr")
data(decathlon)
colnames(decathlon)
glimpse(decathlon)
summary(decathlon)




### Pairwise analysis

1. We may plot the two first coordinates (__100m__ and __Long.jump__) adding the __Points__ information as a color.



In [ ]:
library("ggplot2")
decathlon2 <- decathlon
names(decathlon2) <- paste("P", names(decathlon), sep = "") #Fix for the column names starting by a number...
ggplot(data= decathlon2, aes(x = P100m, y = PLong.jump, color = PPoints)) + geom_point(size = 5) +
  geom_text(label = row.names(decathlon), vjust = -1.25)
  guides(color = FALSE)



Do you think those values are correlated?

2. Let's have a first look on the data by computing the correlation matrix between the 10 first variables.



In [ ]:
cor(decathlon[,1:10])



Can you pick the most correlated variables and the least ones?

3. We may look at the pairwise scatterplot to confirm those findings.



In [ ]:
library(GGally)
ggpairs(decathlon2[,1:10])
ggpairs(decathlon2, columns = c(1,2,6,10))



### 3D plot

Now that we have visualize, our dataset in 2D we can try to explore it in 3D using the __rgl__ package.

4. Use `plot3d` to display the 3D scatterplot of the first three variables. The plot is interactive and you can play with it.



In [ ]:
library(rgl)
library(scales)
plot3d(as.matrix(decathlon[,1:3]), type = "s", size = 5,
       col = cscale(decathlon$Points,seq_gradient_pal("#132B43", high = "#56B1F7")))



Can you find the projection that appears to lose the less information?

5. This task is similar to the PCA where we look a the subspace that minimizes the error between the data and its projection. We can draw the ellipse corresponding to the eigenvectors and the eigenvalues of the covariance matrix to verify this.



In [ ]:
plot3d(as.matrix(decathlon[,1:3]), type = "s", size = 5,
       col = cscale(decathlon$Points,seq_gradient_pal("#132B43", high = "#56B1F7")))
plot3d(ellipse3d(cov(decathlon[,1:3]), centre = colMeans(decathlon[,1:3])), col="grey", alpha=0.25, add = TRUE)
par3d(FOV=1)



Can you verify that the best subspace is defined by the direction of the eigenvectors with the largest eigenvalues?

### Scaling?

So far, we have use the raw performance. That is we compare duration with length as well as 100m time with 1500m time. Obviously, this is not a good idea. All those values should be measured in a common scale. The most classical technique is to _rescale_ by substracting its mean and dividing by the standard deviation. We may repeat all the previous experiments with those rescaled data.



In [ ]:
decathlonR <- decathlon
decathlonR[1:10] <- scale(decathlonR[1:10])
decathlonR2 <- decathlonR
names(decathlonR2) <- paste("P", names(decathlon), sep = "") #Fix for the column names starting by a number...
ggplot(data= decathlonR2, aes(x = P100m, y = PLong.jump, color = PPoints)) + geom_point(size = 5) +
  geom_text(label = row.names(decathlon), vjust = -1.25) +
  guides(color = FALSE)


In [ ]:
cor(decathlonR[,1:10])


In [ ]:
ggpairs(decathlonR2[,1:10])
ggpairs(decathlonR2, columns = c(1,2,6,10))


In [ ]:
plot3d(as.matrix(decathlonR[,1:3]), type = "s", size = 5,
       col = cscale(decathlon$Points,seq_gradient_pal("#132B43", high = "#56B1F7")))
plot3d(ellipse3d(cov(decathlonR[,1:3]), centre = colMeans(decathlonR[,1:3])), col="grey", alpha=0.25, add = TRUE)
par3d(FOV=1)



6. Which results are similar and which are different? Why?

### Principal Component Analysis

7. The package __FactoMineR__ contains a PCA function that we are going to use.


In [ ]:
library("FactoMineR")
PCADecathlonR <- PCA(decathlonR[1:10], graph = FALSE)
str(PCADecathlonR)



Let's look at those results:

- __eig__ contains the eigenvalues (as well as the percentage of variance and the cumulative variance)
- __ind__ contains the new coordinates $V^{(l)t} \textbf{X}$ (as well as related information)
- __var__ contains the projection of the original axis in the new ones (as well as related information)
- __svd__ contains the SVD decomposition of $\textbf{X}_{(n)}$ ($\textbf{X}_{(n)} = U D(\lambda) V^t$)

Can you check that the new coordinates are given by $U D(\lambda)$  and the new basis vectors by $V D(\lambda)$?

7. For those interested, can you recover the PCA using only linear algebra?



In [ ]:
CDecathlonR <- cov(as.matrix(decathlonR[1:10]))
EDecathlonR <- eigen(CDecathlonR)
UDecathlonR <- EDecathlonR$vectors
LambdaDecathlonR <- EDecathlonR$values

PCADecathlonR2 <- as.matrix(decathlonR[1:10]) %*% UDecathlonR



8. Plot the points in the new coordinates



In [ ]:
ggplot(data= data.frame(X1 = PCADecathlonR$ind$coord[,1], X2 = PCADecathlonR$ind$coord[,2], Col = decathlon$Points),
       aes(x = X1, y = X2, color = Col)) + geom_point(size = 5) +
  geom_text(label = row.names(decathlon), vjust = -1.25) +
  scale_x_continuous(expand = c(.15,0)) + scale_y_continuous(expand = c(.1,0)) +
  guides(color = FALSE)



9. We are now interested in the interpretation of the axis themselves. We can compute the projection of those axis on the new ones, the ones made from the new coordinates $U D(\lambda)$. By construction, they are orthogonomal but no necessarily normalized. This however the case for $U$ so that the projection  of $\textbf{X}_{(n)}$ is given by $P_{(n)} = U^t \textbf{X}_{(n)} U$

Verify that indeed the __coords__ of var are given by $U^t \textbf{X}_{(n)}$ up to a scaling...



In [ ]:
t(PCADecathlonR$svd$U) %*% as.matrix(decathlonR[,1:10]) / 40
PCADecathlonR$var$coord



Note that because, we use a _rescaled_ version of we know that all the coordinates have a variance $1$,
we can plot them in the plane spanned by the two first vectors and verify that they remains within the unit circle. The variables that are well represented within this plane would be close to reach the circle whereas the badly represented ones will remain close to $0$.



In [ ]:
circle <- function(center = c(0, 0), npoints = 100) {
    r = 1
    tt = seq(0, 2 * pi, length = npoints)
    xx = center[1] + r * cos(tt)
    yy = center[1] + r * sin(tt)
    return(data.frame(X1 = xx, X2 = yy))
}
corcir = circle(c(0, 0), npoints = 100)

ggplot(data.frame(X1 = PCADecathlonR$var$coord[,1], X2 = PCADecathlonR$var$coord[,2]))+
  geom_path(data = corcir, aes(x = X1, y = X2), color = "gray") +
  geom_segment(aes(xend = X1, yend = X2), x = 0 , y = 0, arrow = grid::arrow()) +
  geom_text(aes(x = X1, y = X2), label = names(decathlonR[1:10]), vjust = -1.25)



Which variables are well captured? Can you interpret the new axes (the horizontal and the vertical ones)?
Can you explain why the long jump appears to be the opposite of the 100m?

10. Now, we may use PCA without the `graph = FALSE` option... and be able to understand the graphs.



In [ ]:
PCADecathlonR <- PCA(decathlonR[1:10])




11. We may check whether we need more dimensions by looking at the eigenvalues or rather the cumulative percentage of variance.



In [ ]:
ggplot(data = data.frame(dim = factor(1:length(PCADecathlonR$eig$`cumulative percentage of variance`)),
                         cum = PCADecathlonR$eig$`cumulative percentage of variance`), aes(x = dim, y = cum)) +
  geom_bar(stat = 'identity')



Do you think 2 dimensions is enough here?

## Multiple Factor Analysis

We consider an extension of the PCA in which one can use qualitative variable and also group variables.

### Data

We will use a dataset made of a sample of 21 red wine from the Loire region. Several attributes are recorded

- 2 qualitative ones:
  + __Label__: _Saumur_, _Chinon_, _Bourgueil_
  + __Soil__: _Reference_, _Env1_, _Env2_, _Env4_
- 29 quantitative ones classified by groups:
  + Odor (5 variables)
  + Visual (3 variables)
  + Odor after shaking (10 variables)
  + Taste (9 variables)
  + Ocerall (2 variables)



In [ ]:
data(wine)
colnames(wine)
glimpse(wine)



### MFA

10. Can you guess what MFA is doing?



In [ ]:
res.mfa = MFA(wine, group = c(2, 5, 3, 10, 9, 2), type = c("n", rep("s",5)), ncp = 5,
              name.group = c("origin", "odor", "visual", "odor.after.shaking", "taste", "overall"),
              num.group.sup=c(1,6))




11. The _strange_ lines in the individual factor map is an attempt to view the contribution of each group. It corresponds to the barycenter of the contributions of the given group. By default, only the two most dispersed and concentrated groups are plotted. We can ask MFA to plot them all.



In [ ]:
plot(res.mfa, choix = "ind", partial = "all")



## MDS

We will apply the MultiDimensional Scaling available in __R__ `cmdscale` to an interesting dataset of the monthly average temperature for 35 cities in Europe. We will try to apply the MDS methodology to visualize those cities in a plane according to their temperature profiles.

12. Read the data and apply the MDS algorithm.



In [ ]:
temperature <- read.csv("temperature.csv", row.names = 1)
temperature[1:12] <- scale(temperature[1:12])

DistTemperature <- dist(temperature[1:12])
TemperatureMDS <- cmdscale(DistTemperature)

ggplot(data = data.frame(X1 = TemperatureMDS[,1], X2 = TemperatureMDS[,2]), aes(x = X1, y = X2)) +
  geom_point() + geom_text(label = row.names(temperature), vjust = -1.25)



13. Can you obtain a better fit with the geography by swapping the axis?



In [ ]:
ggplot(data = data.frame(X1 = TemperatureMDS[,1], X2 = TemperatureMDS[,2]), aes(x = -X2, y = X1)) +
  geom_point() + geom_text(label = row.names(temperature), vjust = -1.25)



14. Compare with the results obtain with the PCA. Is this a suprise?



In [ ]:
TemperaturePCA <- PCA(temperature[1:12], graph = FALSE)
ggplot(data = data.frame(X1 = TemperaturePCA$ind$coord[,1], X2 = TemperaturePCA$ind$coord[,2]), aes(x = X1, y = X2)) +
  geom_point() + geom_text(label = row.names(temperature), vjust = -1.25)

ggplot(data = data.frame(X1 = TemperaturePCA$ind$coord[,1], X2 = TemperaturePCA$ind$coord[,2]), aes(x = X2, y = -X1)) +
  geom_point() + geom_text(label = row.names(temperature), vjust = -1.25)




15. Use the __igraph__ package to perform a graph Laplacian based dimension reduction.
